# GPX Viewer

This app lets you to display a track from a GPX or CSV file recorded with a GPS device.

In [1]:
import sys, os, io
from ipywidgets import Button, HTML, HBox, FileUpload, Label, Output, Layout, Image

sys.path.append(os.getcwd() + '/src')

from sailing_session import SailingSession
from sailing_graphs import *
from utils import calculate_initial_compass_bearing


In [2]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'./img/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [3]:
# Create outputs for map and polar to render into
# call them later to actually display them
global_map_out = Output()
global_polar_out = Output()

In [4]:
def map_polar_buttons(session, map_out=global_map_out, polar_out=global_polar_out):
    def map_button():
        @map_out.capture()
        def map_button_clicked(change):
            map_out.clear_output()
            with map_out:
                chart = ipyleaflet_chart(session)
                graph = bqplot_graph(session)
                link_chart_graph(chart, graph, session)
                display(chart)
                display(graph)

        button = Button(description="Show Map")
        button.on_click(map_button_clicked)
        return button

    def polar_button():
        @polar_out.capture()
        def polar_button_clicked(change):
            polar_out.clear_output()
            with polar_out:
                render_polar(session)

        button = Button(description="Show Polar")
        button.on_click(polar_button_clicked)
        return button
    
    buttons = [map_button(), polar_button()]
    display(HBox(buttons, layout=Layout(flex_flow='row', align_items='center')))

In [5]:
# TODO the gpx examples currently won't show the map because of timestamp issues
# so display CSVs only

def show_examples(output):
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.csv')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @output.capture()
        def on_example_clicked(change):
            # output.clear_output()
            with output:
                display(HBox([Label(value='Loading...')], layout=Layout(flex_flow='row', align_items='center')))
                global session
                session = SailingSession(filename)
                map_polar_buttons(session)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button
    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

In [6]:
def show_uploader(output):
    uploader = FileUpload(accept='.gpx, .csv', multiple=False)

    def handle_upload(change):
        with output:
            display(HBox([Label(value='Loading...')], layout=Layout(flex_flow='row', align_items='center')))
        with open("tmp/session.csv", "w+b") as f:
            f.write(uploader.data[0])
        with output:
            global session
            session = SailingSession("tmp/session.csv")
            map_polar_buttons(session)
        
    uploader.observe(handle_upload, names='value')
    display(uploader)


### Upload File

If you have a csv of a GPS trace, upload it below:

In [7]:
upload_button_out = Output()
show_uploader(upload_button_out)
# call output to display
upload_button_out

FileUpload(value={}, accept='.gpx, .csv', description='Upload')

Output()

### Example

If you don't have a GPX or CSV file, try with one of the following examples:

In [8]:
example_buttons_out = Output()
show_examples(example_buttons_out)
# call the output to display it
example_buttons_out

Output()

In [9]:
global_polar_out

Output()

In [10]:
global_map_out

Output()